In [8]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

sns.set(style="whitegrid", context="notebook", font_scale=1.1)
start_time = time.time()

print("Libraries imported.")

Libraries imported.


In [9]:
# Adjust these paths as needed
data_folder = "Input"

# Load men's regular season detailed results and teams
m_reg = pd.read_csv(os.path.join(data_folder, "MRegularSeasonDetailedResults.csv"))
MTeams = pd.read_csv(os.path.join(data_folder, "MTeams.csv"))

# Load women's regular season detailed results and teams
w_reg = pd.read_csv(os.path.join(data_folder, "WRegularSeasonDetailedResults.csv"))
WTeams = pd.read_csv(os.path.join(data_folder, "WTeams.csv"))

# Load Massey ratings – we assume it contains a column "Team" and "MAS"
massey_ratings = pd.read_csv("Massey Ratings New.csv")

print("Data loaded. (Men’s and Women’s results, teams, and Massey ratings)")

Data loaded. (Men’s and Women’s results, teams, and Massey ratings)


In [10]:
def compute_team_new_rating(df, team_id, bonus=2):
    """
    Computes the new rating for a team as the average adjusted margin.
    Adjust margins using a bonus: if a team is home, add bonus; if away, subtract bonus.
    
    Parameters:
      df (DataFrame): Game results (for seasons 2019-2023).
      team_id (int): The team ID.
      bonus (float): Points bonus for playing at home.
      
    Returns:
      float: The new rating (average adjusted margin).
    """
    margins = []
    
    # Games where team was the winner.
    df_win = df[df["WTeamID"] == team_id].copy()
    df_win["adj_margin"] = df_win["WScore"] - df_win["LScore"]
    df_win.loc[df_win["WLoc"] == "H", "adj_margin"] += bonus
    df_win.loc[df_win["WLoc"] == "A", "adj_margin"] -= bonus
    margins.extend(df_win["adj_margin"].tolist())
    
    # Games where team was the loser.
    df_loss = df[df["LTeamID"] == team_id].copy()
    df_loss["adj_margin"] = -(df_loss["WScore"] - df_loss["LScore"])
    # For losing teams, if the winning team was home then the loser was away (further hurt),
    # and vice versa.
    df_loss.loc[df_loss["WLoc"] == "H", "adj_margin"] -= bonus  
    df_loss.loc[df_loss["WLoc"] == "A", "adj_margin"] += bonus  
    margins.extend(df_loss["adj_margin"].tolist())
    
    if len(margins) == 0:
        return np.nan
    return np.mean(margins)

# Compute team IDs.
team_ids_m = MTeams["TeamID"].unique()
team_ids_w = WTeams["TeamID"].unique()

# Filter results for seasons 2019-2023.
m_reg_new = m_reg[(m_reg["Season"] >= 2019) & (m_reg["Season"] <= 2023)]
w_reg_new = w_reg[(w_reg["Season"] >= 2019) & (w_reg["Season"] <= 2023)]

# Compute new ratings.
new_rating_men = {tid: compute_team_new_rating(m_reg_new, tid) for tid in team_ids_m}
new_rating_women = {tid: compute_team_new_rating(w_reg_new, tid) for tid in team_ids_w}

print("New ratings computed for men and women (2019-2023).")

New ratings computed for men and women (2019-2023).



## Combine Ratings: Elo, New Rating, and Massey
We assume that Elo ratings are provided (here dummy values of 1500 are used). We then map the Massey ratings (column "MAS")
from the Massey ratings file to our teams and compute a weighted average.

In [14]:
# For simplicity, suppose we already have dictionaries for Elo ratings:
elo_m = {tid: 1500 for tid in team_ids_m}   # Replace with actual Elo ratings
elo_w = {tid: 1500 for tid in team_ids_w}

# Create a mapping from team name to MAS rating.
massey_mapping = dict(zip(massey_ratings["Team"], massey_ratings["MAS"]))

# Map MAS ratings onto the teams DataFrames.
MTeams["CombinedMAS"] = MTeams["TeamName"].map(massey_mapping)
WTeams["CombinedMAS"] = WTeams["TeamName"].map(massey_mapping)

def compute_combined_rating(team_id, gender="M", weight_elo=0.33, weight_new=0.33, weight_massey=0.34):
    if gender == "M":
        rating_elo = elo_m.get(team_id, 1500)
        rating_new = new_rating_men.get(team_id, np.nan)
        mas = MTeams.loc[MTeams["TeamID"] == team_id, "CombinedMAS"]
    else:
        rating_elo = elo_w.get(team_id, 1500)
        rating_new = new_rating_women.get(team_id, np.nan)
        mas = WTeams.loc[WTeams["TeamID"] == team_id, "CombinedMAS"]
    rating_massey = mas.values[0] if not mas.empty else np.nan
    ratings = np.array([rating_elo, rating_new, rating_massey], dtype=float)
    weights = np.array([weight_elo, weight_new, weight_massey])
    valid = ~np.isnan(ratings)
    if valid.sum() == 0:
        return np.nan
    return np.average(ratings[valid], weights=weights[valid])

# Compute combined ratings.
combined_rating_men = {tid: compute_combined_rating(tid, "M") for tid in team_ids_m}
combined_rating_women = {tid: compute_combined_rating(tid, "W") for tid in team_ids_w}

print("Combined ratings computed for men and women.")

Combined ratings computed for men and women.


In [12]:
def prepare_model_data(df, combined_rating_dict, gender="M"):
    """
    Computes feature as the difference in combined ratings.
    Target is the game margin (WScore - LScore).
    """
    X, y = [], []
    for _, row in df.iterrows():
        if gender == "M":
            r_w = combined_rating_men.get(row["WTeamID"], np.nan)
            r_l = combined_rating_men.get(row["LTeamID"], np.nan)
        else:
            r_w = combined_rating_women.get(row["WTeamID"], np.nan)
            r_l = combined_rating_women.get(row["LTeamID"], np.nan)
        if np.isnan(r_w) or np.isnan(r_l):
            continue
        diff = r_w - r_l
        X.append([diff])
        y.append(row["WScore"] - row["LScore"])
    return np.array(X), np.array(y)

# Split data (use 2024 as validation)
train_m = m_reg[m_reg["Season"] < 2024]
val_m   = m_reg[m_reg["Season"] == 2024]
train_w = w_reg[w_reg["Season"] < 2024]
val_w   = w_reg[w_reg["Season"] == 2024]

X_train_m, y_train_m = prepare_model_data(train_m, combined_rating_men, "M")
X_val_m, y_val_m     = prepare_model_data(val_m, combined_rating_men, "M")
X_train_w, y_train_w = prepare_model_data(train_w, combined_rating_women, "W")
X_val_w, y_val_w     = prepare_model_data(val_w, combined_rating_women, "W")

print("Training and validation data prepared for both men and women.")

Training and validation data prepared for both men and women.


In [15]:
# %% [code]
def margin_to_probability(margin, scale=10.0):
    """
    Convert a margin value into a win probability using a logistic transformation.
    
    \[
    p = \frac{1}{1 + \exp(-\frac{\text{margin}}{\text{scale}})}
    \]
    
    Parameters:
      margin (float or array-like): The predicted margin.
      scale (float): Scaling factor to control steepness.
      
    Returns:
      float or array-like: Win probability between 0 and 1.
    """
    return 1.0 / (1 + np.exp(-margin / scale))

def tune_and_evaluate(model_obj, param_dist, X_train, y_train, X_val, y_val, scale=10.0):
    """
    Tune the model using RandomizedSearchCV, then evaluate using RMSE on margin predictions and Brier score
    on win probabilities.
    
    The ground truth win probability is computed as 1 if the true margin is > 0, else 0.
    """
    rs = RandomizedSearchCV(model_obj, param_distributions=param_dist, 
                              n_iter=10, cv=3, scoring="neg_mean_squared_error", 
                              random_state=42, error_score='raise', n_jobs=-1)
    rs.fit(X_train, y_train)
    best_model = rs.best_estimator_
    y_pred = best_model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    # Convert predictions and true margins to probabilities.
    true_prob = (y_val > 0).astype(float)
    pred_prob = margin_to_probability(y_pred, scale=scale)
    brier = np.mean((pred_prob - true_prob)**2)
    
    return best_model, rmse, brier

In [16]:
from scipy.stats import uniform, randint

def build_dnn_model(layers=[Dense(32, activation='relu'), Dropout(0.2), Dense(1)]):
    model = Sequential()
    for layer in layers:
        model.add(layer)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

models_grid = {
    "LinearRegression": {
        "model": LinearRegression(),
        "params": {}
    },
    "Ridge": {
        "model": Ridge(),
        "params": {"alpha": uniform(0.1, 10)}
    },
    "Lasso": {
        "model": Lasso(max_iter=5000),
        "params": {"alpha": uniform(0.001, 1)}
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor(),
        "params": {"max_depth": randint(1, 10), "min_samples_split": randint(2, 10)}
    },
    "RandomForest": {
        "model": RandomForestRegressor(random_state=42, n_jobs=-1),
        "params": {"n_estimators": randint(50, 200), "max_depth": randint(1, 10)}
    },
    "ExtraTrees": {
        "model": ExtraTreesRegressor(random_state=42, n_jobs=-1),
        "params": {"n_estimators": randint(50, 200), "max_depth": randint(1, 10)}
    },
    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {"n_estimators": randint(50, 200), "learning_rate": uniform(0.01, 0.3), "max_depth": randint(1, 10)}
    },
    "XGBoost": {
        "model": xgb.XGBRegressor(tree_method="gpu_hist", random_state=42, n_jobs=-1),
        "params": {"n_estimators": randint(50, 200), "learning_rate": uniform(0.01, 0.3), "max_depth": randint(1, 10)}
    },
    "LightGBM": {
        "model": lgb.LGBMRegressor(device="gpu", random_state=42),
        "params": {"n_estimators": randint(50, 200), "learning_rate": uniform(0.01, 0.3), "num_leaves": randint(20, 50)}
    },
    "CatBoost": {
        "model": cb.CatBoostRegressor(task_type="GPU", verbose=0, random_state=42),
        "params": {"iterations": randint(50, 200), "learning_rate": uniform(0.01, 0.3), "depth": randint(3, 10)}
    },
    "DNN": {
        "model": KerasRegressor(model=build_dnn_model, verbose=0),
        "params": {
            "model__layers": [[Dense(32, activation='relu'), Dropout(0.2), Dense(1)],
                              [Dense(64, activation='relu'), Dropout(0.3), Dense(1)]],
            "batch_size": randint(16, 64),
            "epochs": randint(10, 50)
        }
    }
}

In [ ]:
men_results = {}
men_best_models = {}
print("Tuning Men's models:")
for name, info in models_grid.items():
    try:
        best_model, rmse, brier = tune_and_evaluate(info["model"], info["params"], 
                                                    X_train_m, y_train_m, X_val_m, y_val_m)
        men_results[name] = {"rmse": rmse, "brier": brier}
        men_best_models[name] = best_model
        print(f"Men's {name}: RMSE = {rmse:.4f}, Brier = {brier:.4f}")
    except Exception as e:
        print(f"Tuning men's {name} failed: {e}")

print("\nTuning Women's models:")
women_results = {}
women_best_models = {}
for name, info in models_grid.items():
    try:
        best_model, rmse, brier = tune_and_evaluate(info["model"], info["params"], 
                                                    X_train_w, y_train_w, X_val_w, y_val_w)
        women_results[name] = {"rmse": rmse, "brier": brier}
        women_best_models[name] = best_model
        print(f"Women's {name}: RMSE = {rmse:.4f}, Brier = {brier:.4f}")
    except Exception as e:
        print(f"Tuning women's {name} failed: {e}")

print("\nMen's Model Results:")
print(men_results)
print("\nWomen's Model Results:")
print(women_results)

Tuning Men's models:


C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Men's LinearRegression: RMSE = 9.0281, Brier = 0.0540
Men's Ridge: RMSE = 9.0281, Brier = 0.0540
Men's Lasso: RMSE = 9.0297, Brier = 0.0540
Men's DecisionTree: RMSE = 8.5177, Brier = 0.0562
Men's RandomForest: RMSE = 8.5040, Brier = 0.0563
Men's ExtraTrees: RMSE = 8.5953, Brier = 0.0556
Men's GradientBoosting: RMSE = 8.5363, Brier = 0.0560


C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\xgboost\core.py:158: UserWarning: [22:05:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the 

Men's XGBoost: RMSE = 8.5209, Brier = 0.0562
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 107634, number of used features: 1
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1 dense feature groups (0.41 MB) transferred to GPU in 0.000936 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 11.965123
Men's LightGBM: RMSE = 8.5218, Brier = 0.0562


In [ ]:
# For men:
val_preds_m = pd.DataFrame({"ID": np.arange(len(X_val_m))})
for name, model in men_best_models.items():
    val_preds_m[name] = model.predict(X_val_m)

# For women:
val_preds_w = pd.DataFrame({"ID": np.arange(len(X_val_w))})
for name, model in women_best_models.items():
    val_preds_w[name] = model.predict(X_val_w)

print("Validation predictions generated for both men and women.")

In [ ]:
# %% [code]
# For demonstration we simulate a small submission DataFrame.
# In practice, load your actual submission file.
df_sub = pd.DataFrame({
    "ID": ["2024_1101_1102", "2024_3101_3102"],
    "Season": [2024, 2024],
    "Team1": [1101, 3101],
    "Team2": [1102, 3102]
})

def predict_match(row, men_model_name, women_model_name):
    s = row["Season"]
    t1 = row["Team1"]
    t2 = row["Team2"]
    # Use corresponding model predictions (here we use our tuned models on validation)
    # In production, you would use the models to predict on your test features.
    if t1 < 2000 and t2 < 2000:
        # Use men’s prediction: we recompute feature diff using combined ratings.
        diff = combined_rating_men.get(t1, 1500) - combined_rating_men.get(t2, 1500)
        pred = men_best_models[men_model_name].predict(np.array([[diff]]))[0]
    elif t1 >= 3000 and t2 >= 3000:
        diff = combined_rating_women.get(t1, 1500) - combined_rating_women.get(t2, 1500)
        pred = women_best_models[women_model_name].predict(np.array([[diff]]))[0]
    else:
        pred = 0.5
    return pred

# Create submission files for every combination.
submission_folder = "combined_submissions"
os.makedirs(submission_folder, exist_ok=True)
combined_submission_files = []  # keep track of file names

for m_name in men_best_models.keys():
    for w_name in women_best_models.keys():
        df_sub["Pred"] = df_sub.apply(lambda row: predict_match(row, m_name, w_name), axis=1)
        fname = os.path.join(submission_folder, f"submission_{m_name}_{w_name}.csv")
        df_sub[["ID", "Pred"]].to_csv(fname, index=False)
        combined_submission_files.append(fname)
        print(f"Saved submission file: {fname}")

In [ ]:
# %% [code]
# For demonstration we simulate combined_preds_df.
# In a real scenario, you would load all combined submission files (using pd.read_csv) 
# and align them by the "ID" order of the validation matches.
# Here we assume that combined_preds_df has one column per combined submission.
combined_preds_df = pd.DataFrame()
for fname in combined_submission_files:
    # Simulate predictions; in practice, load and merge on ID.
    temp = pd.read_csv(fname)
    col_name = os.path.basename(fname).replace("submission_", "").replace(".csv", "")
    combined_preds_df[col_name] = temp["Pred"]

# Define a function to compute ensemble RMSE and Brier score.
def compute_ensemble_scores(pred_df, y_true, model_list, method="simple", combined_rmse_dict=None):
    if method == "simple":
        ens_pred = pred_df[model_list].mean(axis=1)
    elif method == "weighted":
        # Use inverse RMSE weights (assume combined_rmse_dict contains each model's combined RMSE)
        weights = np.array([1/combined_rmse_dict[m] for m in model_list])
        weights = weights / np.sum(weights)
        ens_pred = np.sum(pred_df[model_list].values * weights, axis=1)
    else:
        raise ValueError("Method not supported.")
    rmse = np.sqrt(mean_squared_error(y_true, ens_pred))
    brier = np.mean((ens_pred - 1)**2)
    return rmse, brier

# For illustration, let’s create a dummy combined_rmse_dict by averaging the men's and women's RMSE.
combined_rmse_dict = {}
for m in men_results:
    for w in women_results:
        key = f"{m}_{w}"
        combined_rmse_dict[key] = (men_results[m]["rmse"] + women_results[w]["rmse"]) / 2

# Sort the combined models by RMSE.
sorted_combined = sorted(combined_rmse_dict.items(), key=lambda x: x[1])
sorted_combined_names = [x[0] for x in sorted_combined]

ensemble_sizes = [2, 3, 5, 7, 9, 11]
ensemble_results = []
for size in ensemble_sizes:
    models_used = sorted_combined_names[:size]
    rmse_simple, brier_simple = compute_ensemble_scores(combined_preds_df, 
                                                        np.ones(len(combined_preds_df)), 
                                                        models_used, method="simple")
    rmse_weighted, brier_weighted = compute_ensemble_scores(combined_preds_df, 
                                                            np.ones(len(combined_preds_df)), 
                                                            models_used, method="weighted",
                                                            combined_rmse_dict=combined_rmse_dict)
    ensemble_results.append({
        "Ensemble": f"top{size}",
        "Method": "simple",
        "RMSE": rmse_simple,
        "Brier": brier_simple
    })
    ensemble_results.append({
        "Ensemble": f"top{size}",
        "Method": "weighted",
        "RMSE": rmse_weighted,
        "Brier": brier_weighted
    })

ensemble_scores_df = pd.DataFrame(ensemble_results)
print("Ensemble performance on combined predictions:")
print(ensemble_scores_df)

# Plot ensemble RMSE vs ensemble size.
plt.figure(figsize=(8,6))
for method in ["simple", "weighted"]:
    subset = ensemble_scores_df[ensemble_scores_df["Method"]==method]
    sizes = [int(x.replace("top","")) for x in subset["Ensemble"]]
    plt.plot(sizes, subset["RMSE"], marker='o', label=method)
plt.xlabel("Number of top combined models in ensemble")
plt.ylabel("RMSE")
plt.title("Ensemble Performance on Combined Predictions")
plt.legend()
plt.show()

In [ ]:
# %% [code]
# Suppose we decide the best ensemble is the weighted ensemble with top3 models.
best_ensemble = sorted_combined_names[:3]
print("Best ensemble models (weighted, top3):", best_ensemble)

# Compute final ensemble prediction on the test set.
# (In production, you would load your test set features, compute rating differences etc.)
# For demonstration, we use our combined_preds_df (assuming it comes from the test set).
final_weights = np.array([1/combined_rmse_dict[m] for m in best_ensemble])
final_weights = final_weights / np.sum(final_weights)
final_preds = np.sum(combined_preds_df[best_ensemble].values * final_weights, axis=1)

# Create final submission DataFrame (assuming same order as df_sub_test)
df_sub_test = pd.read_csv(os.path.join(data_folder, "SampleSubmissionStage2.csv"))
df_sub_test["Pred"] = final_preds
submission_filename = "final_submission_top3_weighted.csv"
df_sub_test[["ID", "Pred"]].to_csv(submission_filename, index=False)
print(f"Final submission saved as {submission_filename}")